~기 + 형용사
~에 + 형용사

In [2]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re

# 데이터 로드
file_path = "merged_data_without_duplicates.csv"
data = pd.read_csv(file_path)

# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 데이터 전처리: "리뷰 없음" 제거
data = data[data["review_content"] != "리뷰 없음"]


# 리뷰 데이터 전처리 함수
def preprocess_review(review):
    # 한글과 공백만 남기기
    return re.sub(r"[^가-힣\s]", "", str(review))


# 특정 패턴 추출 함수 (예: ~기 좋은)
def extract_meaningful_phrases(text):
    tokens = okt.pos(text)
    phrases = []
    for i in range(len(tokens) - 2):
        # 형식: 동사 + '기' + 형용사 (ex: 읽기 좋은)
        if (
            tokens[i][1] == "Verb"
            and tokens[i + 1][0] == "기"
            and tokens[i + 2][1] == "Adjective"
        ):
            phrases.append(f"{tokens[i][0]}기 {tokens[i + 2][0]}")
        # 형식: 명사 + '에' + 형용사 (ex: 자기 전에 좋은)
        if (
            tokens[i][1] == "Noun"
            and tokens[i + 1][0] == "에"
            and tokens[i + 2][1] == "Adjective"
        ):
            phrases.append(f"{tokens[i][0]}에 {tokens[i + 2][0]}")
    return phrases


# 데이터 전처리 및 키워드 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["keywords"] = data["cleaned_review"].apply(extract_meaningful_phrases)

# 모든 리뷰에서 추출된 키워드 리스트 생성
all_phrases = [phrase for sublist in data["keywords"] for phrase in sublist]

# 키워드 빈도 계산
phrase_counts = Counter(all_phrases)

# 자주 사용된 키워드 추출 (상위 20개)
common_phrases = phrase_counts.most_common(20)

# 결과 출력
print("Top 20 Most Common Meaningful Phrases:")
for phrase, count in common_phrases:
    print(f"Phrase: {phrase}, Count: {count}")



Top 20 Most Common Meaningful Phrases:
Phrase: 책에 있는, Count: 44
Phrase: 만에 좋은, Count: 27
Phrase: 집에 있는, Count: 26
Phrase: 이번에 새로, Count: 21
Phrase: 안에 있는, Count: 20
Phrase: 옆에 있는, Count: 17
Phrase: 시대에 필요한, Count: 16
Phrase: 보기에 좋은, Count: 15
Phrase: 곁에 있는, Count: 15
Phrase: 덕분에 좋은, Count: 14
Phrase: 기본에 충실한, Count: 12
Phrase: 삶에 많은, Count: 11
Phrase: 뒤에 있는, Count: 11
Phrase: 중에 이런, Count: 11
Phrase: 시대에 이런, Count: 10
Phrase: 공부에 많은, Count: 10
Phrase: 임에 틀림없다, Count: 10
Phrase: 속에 있는, Count: 9
Phrase: 함에 있어서, Count: 9
Phrase: 간만에 좋은, Count: 9


n-gram(3~5)패턴 추출

In [5]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re

# 데이터 로드
file_path = "merged_data_without_duplicates.csv"
data = pd.read_csv(file_path)

# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 데이터 전처리: "리뷰 없음" 제거
data = data[data["review_content"] != "리뷰 없음"]

# 리뷰 데이터 전처리 함수
def preprocess_review(review):
    # 한글과 공백만 남기기
    return re.sub(r"[^가-힣\s]", "", str(review))

# n-gram 추출 함수
def extract_phrases(text, n=4):
    tokens = okt.morphs(text)  # 형태소 단위로 분리
    ngrams = [
        " ".join(tokens[i : i + n]) for i in range(len(tokens) - n + 1)
    ]  # n-gram 생성
    return ngrams

# 데이터 전처리 및 n-gram 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["phrases"] = data["cleaned_review"].apply(lambda x: extract_phrases(x, n=4))  # 4-gram 추출

# 모든 리뷰에서 추출된 키워드 리스트 생성
all_phrases = [phrase for sublist in data["phrases"] for phrase in sublist]

# 키워드 빈도 계산
phrase_counts = Counter(all_phrases)

# 자주 사용된 키워드 추출 (상위 20개)
common_phrases = phrase_counts.most_common(20)

# 결과 출력
print("Top 20 Most Common Phrases:")
for phrase, count in common_phrases:
    print(f"Phrase: {phrase}, Count: {count}")

# CSV 파일로 저장 (옵션)
# output_file = "/mnt/data/common_phrases.csv"
# pd.DataFrame(common_phrases, columns=["Phrase", "Count"]).to_csv(output_file, index=False)
# print(f"Results saved to {output_file}")


Top 20 Most Common Phrases:
Phrase: 을 확인 할 수, Count: 624
Phrase: 내용 을 확인 할, Count: 606
Phrase: 확인 할 수 있습니다, Count: 606
Phrase: 이 글 에는 스포일러, Count: 604
Phrase: 글 에는 스포일러 가, Count: 604
Phrase: 에는 스포일러 가 포함, Count: 604
Phrase: 스포일러 가 포함 되어, Count: 604
Phrase: 가 포함 되어 있습니다, Count: 604
Phrase: 포함 되어 있습니다 보시겠습니까, Count: 604
Phrase: 되어 있습니다 보시겠습니까 회색, Count: 604
Phrase: 있습니다 보시겠습니까 회색 영역, Count: 604
Phrase: 보시겠습니까 회색 영역 을, Count: 604
Phrase: 회색 영역 을 클릭, Count: 604
Phrase: 영역 을 클릭 하면, Count: 604
Phrase: 을 클릭 하면 내용, Count: 604
Phrase: 클릭 하면 내용 을, Count: 604
Phrase: 하면 내용 을 확인, Count: 604
Phrase: 이 책 을 읽고, Count: 439
Phrase: 이 책 을 통해, Count: 309
Phrase: 마음 에 듭니 다, Count: 207


In [6]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re

# 데이터 로드
file_path = "merged_data_without_duplicates.csv"
data = pd.read_csv(file_path)

# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 데이터 전처리: "리뷰 없음" 제거
data = data[data["review_content"] != "리뷰 없음"]

# 불용어 리스트 정의 (단어 기준)
stop_words = ["확인", "포함", "내용", "보시겠습니까", "스포일러", "되어", "있습니다", "클릭", "영역"]

# 리뷰 데이터 전처리 함수
def preprocess_review(review):
    # 한글과 공백만 남기기
    return re.sub(r"[^가-힣\s]", "", str(review))

# n-gram 추출 함수
def extract_phrases(text, n=4):
    tokens = okt.morphs(text)  # 형태소 단위로 분리
    ngrams = [
        " ".join(tokens[i : i + n]) for i in range(len(tokens) - n + 1)
    ]  # n-gram 생성
    return ngrams

# 불용어 필터링 함수
def filter_phrases(phrases):
    return [
        phrase for phrase in phrases
        if not any(stop_word in phrase for stop_word in stop_words)
    ]

# 데이터 전처리 및 n-gram 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["phrases"] = data["cleaned_review"].apply(lambda x: extract_phrases(x, n=4))  # 4-gram 생성
data["filtered_phrases"] = data["phrases"].apply(filter_phrases)

# 모든 리뷰에서 추출된 키워드 리스트 생성
all_phrases = [phrase for sublist in data["filtered_phrases"] for phrase in sublist]

# 키워드 빈도 계산
phrase_counts = Counter(all_phrases)

# 자주 사용된 키워드 추출 (상위 20개)
common_phrases = phrase_counts.most_common(20)

# 결과 출력
print("Top 20 Most Common Meaningful Phrases:")
for phrase, count in common_phrases:
    print(f"Phrase: {phrase}, Count: {count}")

# CSV 파일로 저장 (옵션)
# output_file = "/mnt/data/filtered_phrases.csv"
# pd.DataFrame(common_phrases, columns=["Phrase", "Count"]).to_csv(output_file, index=False)
# print(f"Results saved to {output_file}")


Top 20 Most Common Meaningful Phrases:
Phrase: 이 책 을 읽고, Count: 439
Phrase: 이 책 을 통해, Count: 309
Phrase: 마음 에 듭니 다, Count: 207
Phrase: 도움 이 될 것, Count: 189
Phrase: 읽어 볼 만 한, Count: 166
Phrase: 생각 이 듭니 다, Count: 163
Phrase: 볼 만 한 책, Count: 161
Phrase: 많은 사람 들 이, Count: 160
Phrase: 할 수 있을 것, Count: 153
Phrase: 책 이라고 생각 합니다, Count: 153
Phrase: 할 수 있는 책, Count: 143
Phrase: 책 인 것 같습니다, Count: 142
Phrase: 들 을 위 한, Count: 131
Phrase: 읽을 수 있는 책, Count: 130
Phrase: 이 책 한 권, Count: 128
Phrase: 도움 이 되는 책, Count: 126
Phrase: 많은 생각 을 하게, Count: 125
Phrase: 처음 부터 끝 까지, Count: 119
Phrase: 작가 님 의 책, Count: 119
Phrase: 이 이 책 을, Count: 115


In [8]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re

# 데이터 로드
file_path = "merged_data_without_duplicates.csv"
data = pd.read_csv(file_path)

# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 데이터 전처리: "리뷰 없음" 제거
data = data[data["review_content"] != "리뷰 없음"]

# 불용어 리스트 정의 (문구 기준)
stop_phrases = [
    "이 책", "마음 에 듭니 다", "생각 이 듭니 다", "할 수 있는 책",
    "많은 사람 들 이", "책 인 것 같습니다", "이 책 을 통해", 
    "책 이라고 생각 합니다", "이 책 한 권", "작가 님 의 책",
    "도움 이 될 것", "처음 부터 끝 까지", "들 을 위 한",
    "할 수 있을 것", "읽을 수 있는 책", "볼 만 한 책",
    "읽어 볼 만 한", "이 이 책 을", "많은 생각 을 하게","확인",
    "포함", "내용", "보시겠습니까", "스포일러", "되어", "있습니다", "클릭", "영역"
]

# 리뷰 데이터 전처리 함수
def preprocess_review(review):
    # 한글과 공백만 남기기
    return re.sub(r"[^가-힣\s]", "", str(review))

# n-gram 추출 함수
def extract_phrases(text, n=4):
    tokens = okt.morphs(text)  # 형태소 단위로 분리
    ngrams = [
        " ".join(tokens[i : i + n]) for i in range(len(tokens) - n + 1)
    ]  # n-gram 생성
    return ngrams

# 불용어 필터링 함수
def filter_phrases(phrases):
    return [
        phrase for phrase in phrases
        if phrase not in stop_phrases
    ]

# 데이터 전처리 및 n-gram 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["phrases"] = data["cleaned_review"].apply(lambda x: extract_phrases(x, n=4))  # 4-gram 생성
data["filtered_phrases"] = data["phrases"].apply(filter_phrases)

# 모든 리뷰에서 추출된 키워드 리스트 생성
all_phrases = [phrase for sublist in data["filtered_phrases"] for phrase in sublist]

# 키워드 빈도 계산
phrase_counts = Counter(all_phrases)

# 자주 사용된 키워드 추출 (상위 20개)
common_phrases = phrase_counts.most_common(20)

# 결과 출력
print("Top 20 Most Common Meaningful Phrases (Excluding Stop Phrases):")
for phrase, count in common_phrases:
    print(f"Phrase: {phrase}, Count: {count}")

# # CSV 파일로 저장 (옵션)
# output_file = "/mnt/data/filtered_meaningful_phrases.csv"
# pd.DataFrame(common_phrases, columns=["Phrase", "Count"]).to_csv(output_file, index=False)
# print(f"Results saved to {output_file}")


Top 20 Most Common Meaningful Phrases (Excluding Stop Phrases):
Phrase: 을 확인 할 수, Count: 624
Phrase: 내용 을 확인 할, Count: 606
Phrase: 확인 할 수 있습니다, Count: 606
Phrase: 이 글 에는 스포일러, Count: 604
Phrase: 글 에는 스포일러 가, Count: 604
Phrase: 에는 스포일러 가 포함, Count: 604
Phrase: 스포일러 가 포함 되어, Count: 604
Phrase: 가 포함 되어 있습니다, Count: 604
Phrase: 포함 되어 있습니다 보시겠습니까, Count: 604
Phrase: 되어 있습니다 보시겠습니까 회색, Count: 604
Phrase: 있습니다 보시겠습니까 회색 영역, Count: 604
Phrase: 보시겠습니까 회색 영역 을, Count: 604
Phrase: 회색 영역 을 클릭, Count: 604
Phrase: 영역 을 클릭 하면, Count: 604
Phrase: 을 클릭 하면 내용, Count: 604
Phrase: 클릭 하면 내용 을, Count: 604
Phrase: 하면 내용 을 확인, Count: 604
Phrase: 이 책 을 읽고, Count: 439
Phrase: 도움 이 되는 책, Count: 126
Phrase: 에 이 책 을, Count: 113


In [9]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re

# 데이터 로드
file_path = "merged_data_without_duplicates.csv"
data = pd.read_csv(file_path)

# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 데이터 전처리: "리뷰 없음" 제거
data = data[data["review_content"] != "리뷰 없음"]

# 불용어 리스트 정의 (문구 기준, 확장됨)
stop_phrases = [
    "이 책", "을 확인 할 수", "내용 을 확인 할", "확인 할 수 있습니다",
    "이 글 에는 스포일러", "글 에는 스포일러 가", "에는 스포일러 가 포함",
    "스포일러 가 포함 되어", "가 포함 되어 있습니다", "포함 되어 있습니다 보시겠습니까",
    "되어 있습니다 보시겠습니까 회색", "있습니다 보시겠습니까 회색 영역",
    "보시겠습니까 회색 영역 을", "회색 영역 을 클릭", "영역 을 클릭 하면",
    "을 클릭 하면 내용", "클릭 하면 내용 을", "하면 내용 을 확인",
    "마음 에 듭니 다", "생각 이 듭니 다", "도움 이 되는 책", "에 이 책 을",
]

# 리뷰 데이터 전처리 함수
def preprocess_review(review):
    # 한글과 공백만 남기기
    return re.sub(r"[^가-힣\s]", "", str(review))

# n-gram 추출 함수
def extract_phrases(text, n=4):
    tokens = okt.morphs(text)  # 형태소 단위로 분리
    ngrams = [
        " ".join(tokens[i : i + n]) for i in range(len(tokens) - n + 1)
    ]  # n-gram 생성
    return ngrams

# 불용어 필터링 함수
def filter_phrases(phrases):
    return [
        phrase for phrase in phrases
        if phrase not in stop_phrases
    ]

# 데이터 전처리 및 n-gram 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["phrases"] = data["cleaned_review"].apply(lambda x: extract_phrases(x, n=4))  # 4-gram 생성
data["filtered_phrases"] = data["phrases"].apply(filter_phrases)

# 모든 리뷰에서 추출된 키워드 리스트 생성
all_phrases = [phrase for sublist in data["filtered_phrases"] for phrase in sublist]

# 키워드 빈도 계산
phrase_counts = Counter(all_phrases)

# 자주 사용된 키워드 추출 (상위 20개)
common_phrases = phrase_counts.most_common(20)

# 결과 출력
print("Top 20 Most Common Meaningful Phrases (Excluding Stop Phrases):")
for phrase, count in common_phrases:
    print(f"Phrase: {phrase}, Count: {count}")

# # CSV 파일로 저장 (옵션)
# output_file = "filtered_meaningful_phrases.csv"
# pd.DataFrame(common_phrases, columns=["Phrase", "Count"]).to_csv(output_file, index=False)
# print(f"Results saved to {output_file}")


Top 20 Most Common Meaningful Phrases (Excluding Stop Phrases):
Phrase: 이 책 을 읽고, Count: 439
Phrase: 이 책 을 통해, Count: 309
Phrase: 도움 이 될 것, Count: 189
Phrase: 읽어 볼 만 한, Count: 166
Phrase: 볼 만 한 책, Count: 161
Phrase: 많은 사람 들 이, Count: 160
Phrase: 할 수 있을 것, Count: 153
Phrase: 책 이라고 생각 합니다, Count: 153
Phrase: 할 수 있는 책, Count: 143
Phrase: 책 인 것 같습니다, Count: 142
Phrase: 들 을 위 한, Count: 131
Phrase: 읽을 수 있는 책, Count: 130
Phrase: 이 책 한 권, Count: 128
Phrase: 많은 생각 을 하게, Count: 125
Phrase: 처음 부터 끝 까지, Count: 119
Phrase: 작가 님 의 책, Count: 119
Phrase: 이 이 책 을, Count: 115
Phrase: 수 있는 책 입니다, Count: 108
Phrase: 에 관 한 책, Count: 107
Phrase: 많은 분 들 이, Count: 103


In [10]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re

# 데이터 로드
file_path = "merged_data_without_duplicates.csv"
data = pd.read_csv(file_path)

# Okt 형태소 분석기 초기화
okt = Okt()

# 리뷰 데이터 전처리: "리뷰 없음" 제거
data = data[data["review_content"] != "리뷰 없음"]

# 불용어 리스트 정의 (문구 기준, 확장됨)
stop_phrases = [
    "이 책", "을 확인 할 수", "내용 을 확인 할", "확인 할 수 있습니다",
    "이 글 에는 스포일러", "글 에는 스포일러 가", "에는 스포일러 가 포함",
    "스포일러 가 포함 되어", "가 포함 되어 있습니다", "포함 되어 있습니다 보시겠습니까",
    "되어 있습니다 보시겠습니까 회색", "있습니다 보시겠습니까 회색 영역",
    "보시겠습니까 회색 영역 을", "회색 영역 을 클릭", "영역 을 클릭 하면",
    "을 클릭 하면 내용", "클릭 하면 내용 을", "하면 내용 을 확인",
    "마음 에 듭니 다", "생각 이 듭니 다", "도움 이 되는 책", "에 이 책 을",
    "이 책 을 읽고", "이 책 을 통해", "도움 이 될 것", "읽어 볼 만 한",
    "볼 만 한 책", "많은 사람 들 이", "할 수 있을 것", "책 이라고 생각 합니다",
    "할 수 있는 책", "책 인 것 같습니다", "들 을 위 한", "읽을 수 있는 책",
    "이 책 한 권", "많은 생각 을 하게", "처음 부터 끝 까지", "작가 님 의 책",
    "이 이 책 을", "수 있는 책 입니다", "에 관 한 책", "많은 분 들 이"
]

# 리뷰 데이터 전처리 함수
def preprocess_review(review):
    # 한글과 공백만 남기기
    return re.sub(r"[^가-힣\s]", "", str(review))

# n-gram 추출 함수
def extract_phrases(text, n=4):
    tokens = okt.morphs(text)  # 형태소 단위로 분리
    ngrams = [
        " ".join(tokens[i : i + n]) for i in range(len(tokens) - n + 1)
    ]  # n-gram 생성
    return ngrams

# 불용어 필터링 함수
def filter_phrases(phrases):
    return [
        phrase for phrase in phrases
        if phrase not in stop_phrases
    ]

# 데이터 전처리 및 n-gram 추출
data["cleaned_review"] = data["review_content"].apply(preprocess_review)
data["phrases"] = data["cleaned_review"].apply(lambda x: extract_phrases(x, n=4))  # 4-gram 생성
data["filtered_phrases"] = data["phrases"].apply(filter_phrases)

# 모든 리뷰에서 추출된 키워드 리스트 생성
all_phrases = [phrase for sublist in data["filtered_phrases"] for phrase in sublist]

# 키워드 빈도 계산
phrase_counts = Counter(all_phrases)

# 자주 사용된 키워드 추출 (상위 20개)
common_phrases = phrase_counts.most_common(20)

# 결과 출력
print("Top 20 Most Common Meaningful Phrases (Excluding Stop Phrases):")
for phrase, count in common_phrases:
    print(f"Phrase: {phrase}, Count: {count}")

# # CSV 파일로 저장 (옵션)
# output_file = "filtered_meaningful_phrases.csv"
# pd.DataFrame(common_phrases, columns=["Phrase", "Count"]).to_csv(output_file, index=False)
# print(f"Results saved to {output_file}")


Top 20 Most Common Meaningful Phrases (Excluding Stop Phrases):
Phrase: 한 권 의 책, Count: 102
Phrase: 이해 할 수 있는, Count: 99
Phrase: 좋은 책 인 것, Count: 98
Phrase: 수 있을 것 같아요, Count: 98
Phrase: 시간 가는 줄 모르고, Count: 98
Phrase: 책 인 것 같아요, Count: 97
Phrase: 가치 가 있는 책, Count: 96
Phrase: 이 책 을 읽으면서, Count: 94
Phrase: 분 들 께 추천, Count: 92
Phrase: 님 의 책 을, Count: 91
Phrase: 가 이 책 을, Count: 90
Phrase: 는 이 책 을, Count: 88
Phrase: 에 관 한 이야기, Count: 87
Phrase: 들 이 이 책, Count: 86
Phrase: 읽어 봐야 할 책, Count: 85
Phrase: 맘 에 듭니 다, Count: 84
Phrase: 책 한 권 으로, Count: 81
Phrase: 이 책 을 구매, Count: 80
Phrase: 추천 하고 싶은 책, Count: 80
Phrase: 이 될 것 같습니다, Count: 80
